# fastgpu

> API details

In [ ]:
# default_exp core

In [ ]:
#export
import os
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
from fastcore.all import *
from pynvml import *

In [ ]:
#hide
from nbdev.showdoc import *

## Overview

Here's what fastgpu does:

1. poll `to_run`
1. find first file
1. check there's an available worker id
1. move it to `running`
1. handle the script
   1. create lock file
   1. redirect stdout/err to `out`
   1. run it
   1. when done, move it to `complete` or `failed`
   1. unlock

For demonstrating how to use `fastgpu`, we first create a directory to store our scripts and outputs:

In [ ]:
path = Path('data')

In [ ]:
#export
def setup_dirs(path):
    "Create and return the following subdirs of `path`: to_run running complete fail out"
    path.mkdir(exist_ok=True)
    dirs = L(path/o for o in 'to_run running complete fail out'.split())    
    for o in dirs: o.mkdir(exist_ok=True)
    return dirs

These are all the subdirectories that are created for us. Your scripts go in `to_run`.

In [ ]:
path_run,path_running,path_complete,path_fail,path_out = setup_dirs(path)

Let's create a scripts directory with a couple of "scripts" (actually symlinks for this demo) in it.

In [ ]:
import shutil
def _setup_test_env():
    shutil.rmtree('data')
    res = setup_dirs(path)
    os.symlink(Path('test_scripts/script_succ.sh').absolute(), path_run/'script_succ.sh')
    os.symlink(Path('test_scripts/script_fail.sh').absolute(), path_run/'script_fail.sh')
    (path_run/'test_dir').mkdir(exist_ok=True)

In [ ]:
_setup_test_env()

## Helper functions for scripts

These functions are used to find and run scripts, and move scripts to the appropriate subdirectory at the appropriate time.

In [ ]:
#export
def find_next_script(p):
    "Get the first script from `p` (in sorted order)"
    files = p.ls().sorted().filter(Self.is_file())
    if files: return files[0]

In [ ]:
test_eq(find_next_script(path_run).name, 'script_fail.sh')
assert not find_next_script(path_complete)

In [ ]:
#export
def safe_rename(file, dest):
    "Move `file` to `dest`, prefixing a random uuid if there's a name conflict"
    to_name = dest/file.name
    if to_name.exists():
        u = uuid4()
        to_name = dest/f'{name}-{u}'
        warnings.warn(f'Using unique name {to_name}')
    file.replace(to_name)
    return to_name

## ResourcePoolBase -

In [ ]:
#export
class ResourcePoolBase():
    def __init__(self, path):
        self.path = Path(path)
        setup_dirs(self.path)
    
    def _lockpath(self,ident): return self.path/f'{ident}.lock'
    def _is_locked(self,ident): return self._lockpath(ident).exists()
    def lock(self,ident, txt='locked'): self._lockpath(ident).write_text(str(txt))
    def unlock(self,ident): return self._lockpath(ident).unlink() if self._is_locked(ident) else None
    def is_available(self,ident): return not self._is_locked(ident)
    def all_ids(self): raise NotImplementedError
    def find_next(self): return first(o for o in self.all_ids() if self.is_available(o))
    def lock_next(self):
        ident = self.find_next()
        if ident is None: return
        self.lock(ident)
        return ident

    def _launch(self, script, ident, env):
        with (self.path/'out'/f'{script.name}.stderr').open("w") as stderr:
            with (self.path/'out'/f'{script.name}.stdout').open("w") as stdout:
                process = subprocess.Popen(str(script), env=env, stdout=stdout, stderr=stderr)
                self.lock(ident, process.pid)
                return process.wait()

    def _run(self, script, ident):
        failed = False
        env=copy(os.environ)
        env['FASTGPU_ID'] = str(ident)
        try: res = self._launch(script, ident, env=env)
        except Exception as e: failed = str(e)
        (self.path/'out'/f'{script.name}.exitcode').write_text(failed if failed else str(res))
        dest = self.path/'fail' if failed or res else self.path/'complete'
        finish_name = safe_rename(script, dest)
        self.unlock(ident)

    def run(self, *args, **kwargs):
        thread = Thread(target=self._run, args=args, kwargs=kwargs)
        thread.start()

    def poll_scripts(self, poll_interval=0.1, exit_when_empty=True):
        while True:
            sleep(poll_interval)
            script = find_next_script(self.path/'to_run')
            if script is None:
                if exit_when_empty: break
                else: continue
            ident = self.lock_next()
            if ident is None: continue
            run_name = safe_rename(script, self.path/'running')
            self.run(run_name, ident)

In [ ]:
#export
add_docs(ResourcePoolBase, "Base class for locked access to list of idents",
         unlock="Remove lockfile for `ident`",
         lock="Create lockfile for `ident`",
         is_available="Is `ident` available",
         all_ids="All idents (abstract method)",
         find_next="Finds next available resource, or None",
         lock_next="Locks an available resource and returns its ident, or None",
         run="Run `script` using resource `ident`",
         poll_scripts="Poll `to_run` for scripts and run in parallel on available resources")

This abstract class locks and unlocks resources using lockfiles. Override `all_ids` to make the list of resources available. See `FixedWorkerPool` for a simple example and details on each method.

In [ ]:
#export
class FixedWorkerPool(ResourcePoolBase):
    "Vends locked access to fixed list of idents"
    def __init__(self, worker_ids, path):
        super().__init__(path)
        self.worker_ids = worker_ids

    def all_ids(self):
        "All available idents"
        return self.worker_ids

The simplest possible `ResourcePoolBase` subclass - the resources are just a list of ids. For instance:

In [ ]:
_setup_test_env()
wp = FixedWorkerPool(L.range(4), path)

In [ ]:
show_doc(FixedWorkerPool.unlock)

<h4 id="ResourcePoolBase.unlock" class="doc_header"><code>ResourcePoolBase.unlock</code><a href="__main__.py#L10" class="source_link" style="float:right">[source]</a></h4>

> <code>ResourcePoolBase.unlock</code>(**`ident`**)

Remove lockfile for `ident`

If there are no locks, this does nothing:

In [ ]:
wp.unlock(0)

In [ ]:
show_doc(FixedWorkerPool.find_next)

<h4 id="ResourcePoolBase.find_next" class="doc_header"><code>ResourcePoolBase.find_next</code><a href="__main__.py#L13" class="source_link" style="float:right">[source]</a></h4>

> <code>ResourcePoolBase.find_next</code>()

Finds next available resource, or None

Initially all resources are available (unlocked), so the first from the provided list will be returned:

In [ ]:
test_eq(wp.find_next(), 0)

In [ ]:
show_doc(FixedWorkerPool.lock)

<h4 id="ResourcePoolBase.lock" class="doc_header"><code>ResourcePoolBase.lock</code><a href="__main__.py#L9" class="source_link" style="float:right">[source]</a></h4>

> <code>ResourcePoolBase.lock</code>(**`ident`**, **`txt`**=*`'locked'`*)

Create lockfile for `ident`

After locking the first resource, it is no longer returned next:

In [ ]:
wp.lock(0)
test_eq(wp.find_next(), 1)

In [ ]:
show_doc(FixedWorkerPool.lock_next)

<h4 id="ResourcePoolBase.lock_next" class="doc_header"><code>ResourcePoolBase.lock_next</code><a href="__main__.py#L14" class="source_link" style="float:right">[source]</a></h4>

> <code>ResourcePoolBase.lock_next</code>()

Locks an available resource and returns its ident, or None

This is the normal way to access a resource - it simply combines `find_next` and `lock`:

In [ ]:
wp.lock_next()
test_eq(wp.find_next(), 2)

In [ ]:
show_doc(FixedWorkerPool.run)

<h4 id="ResourcePoolBase.run" class="doc_header"><code>ResourcePoolBase.run</code><a href="__main__.py#L39" class="source_link" style="float:right">[source]</a></h4>

> <code>ResourcePoolBase.run</code>(**\*`args`**, **\*\*`kwargs`**)

Run `script` using resource `ident`

In [ ]:
_setup_test_env()
wp = FixedWorkerPool(L.range(4), path)

In [ ]:
_setup_test_env()
f = find_next_script(path_run)
wp._run(f, 0)

test_eq(find_next_script(path_run), path_run/'script_succ.sh')
test_eq((path_out/'script_fail.sh.exitcode').read_text(), '1')
assert (path_fail/'script_fail.sh').exists()

In [ ]:
show_doc(FixedWorkerPool.poll_scripts)

<h4 id="ResourcePoolBase.poll_scripts" class="doc_header"><code>ResourcePoolBase.poll_scripts</code><a href="__main__.py#L43" class="source_link" style="float:right">[source]</a></h4>

> <code>ResourcePoolBase.poll_scripts</code>(**`poll_interval`**=*`0.1`*, **`exit_when_empty`**=*`True`*)

Poll `to_run` for scripts and run in parallel on available resources

In [ ]:
_setup_test_env()
wp.poll_scripts()

assert not find_next_script(path_run), find_next_script(path_run)
test_eq((path_out/'script_fail.sh.exitcode').read_text(), '1')
test_eq((path_out/'script_succ.sh.exitcode').read_text(), '0')
assert not (path_run/'script_fail.sh').exists()
assert (path_fail/'script_fail.sh').exists()
assert (path_complete/'script_succ.sh').exists()
test_eq((path_out/'script_succ.sh.stdout').read_text(), '0\n')

## GPU

In [ ]:
#export
class ResourcePoolGPU(ResourcePoolBase):
    "Vends locked access to NVIDIA GPUs"
    def __init__(self, path):
        super().__init__(path)
        nvmlInit()
        # NVML doesn't respect CUDA_VISIBLE_DEVICES, so we need to query manually
        envvar = os.environ.get("CUDA_VISIBLE_DEVICES", '')
        self.devs = (L(envvar.split(',')).map(int) if envvar!=''
                     else L.range(nvmlDeviceGetCount()))
        self.ids = L.range(self.devs)

    def _launch(self, script, ident, env):
        env['CUDA_VISIBLE_DEVICES'] = str(self.devs[ident])
        return super()._launch(script, ident, env)
#         with (self.path/'out'/f'{script.name}.stderr').open("w") as stderr:
#             with (self.path/'out'/f'{script.name}.stdout').open("w") as stdout:
#                 process = subprocess.Popen(str(script), env=env, stdout=stdout, stderr=stderr)
#                 self.lock(ident, process.pid)
#                 return process.wait()
#         with (self.path/'out'/f'{script.name}.stderr').open("w") as stderr:
#             with (self.path/'out'/f'{script.name}.stdout').open("w") as stdout:
#                 return subprocess.call(str(script), env=env, stdout=stdout, stderr=stderr)

    def is_available(self,ident):
        "If a GPU's used_memory is less than 1G and is running no procs then it will be regarded as available"
        if not super().is_available(ident): return False
        device = nvmlDeviceGetHandleByIndex(self.devs[ident])
        if nvmlDeviceGetComputeRunningProcesses(device): return False
        return nvmlDeviceGetMemoryInfo(device).used <= 1e9

    def all_ids(self):
        "All GPUs"
        return self.ids

In [ ]:
# These only work if you have an NVIDIA GPU installed
# wp = ResourcePoolGPU('data')
# wp.find_next()

This is a resource pool that uses [pynvml](https://pypi.org/project/pynvml/) to find GPUs that aren't being used (based on whether they have memory allocated). It is implemented by overriding two methods from `ResourcePoolBase`. Usage is identical to `FixedWorkerPool`, except that you don't need to pass in `worker_ids`, since available GPUs are considered to be the resource pool.

In [ ]:
show_doc(ResourcePoolGPU.is_available)

<h4 id="ResourcePoolGPU.is_available" class="doc_header"><code>ResourcePoolGPU.is_available</code><a href="__main__.py#L13" class="source_link" style="float:right">[source]</a></h4>

> <code>ResourcePoolGPU.is_available</code>(**`ident`**)

If a GPU's used_memory is less than 1G and is running no procs then it will be regarded as available

In [ ]:
show_doc(ResourcePoolGPU.all_ids)

<h4 id="ResourcePoolGPU.all_ids" class="doc_header"><code>ResourcePoolGPU.all_ids</code><a href="__main__.py#L20" class="source_link" style="float:right">[source]</a></h4>

> <code>ResourcePoolGPU.all_ids</code>()

All GPUs

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_utils.ipynb.
Converted index.ipynb.
